# Santanders Customer Transaction - Competition

I am using this notebook to learn new and advance techniques, before I can begin reading others works, I want to work on the data myself and get a feel of the problem.

* I am looking to understand the limitation this large amount of data
* I am seeing what a base accuracy any novice can get
* I recently learnt upscaling, RFE so I want to test it out
* I am pretty sure, a lot of the improvement will be done based on how the data is preprocessed! So I want to use my own ideas before Begin to read others

Readers, please feel free to comment/ crticize/ advice 

In [ ]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# data = pd.read_csv('Data/train.csv',index_col='ID_code')
data = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/train.csv')

# data.set_index('ID_code',inplace=True)
data.drop('ID_code',axis=1,inplace=True)

### Exploratoty Data Analysis - Basic

How much of a -ve impact will it make if I run the EDA on 

* first 500 or 'x' data
* random 500 or 'x' data

How can I determine 'x'

In [ ]:
data = data.sample(100)

In [ ]:
fig, axs = plt.subplots(1,10,figsize=(30,5))
fig.subplots_adjust(hspace = .5, wspace=.5)
axs = axs.ravel()

counter=0
for i in data.columns[1:10]:
    
    axs[counter].set_title(i)
#     axs[counter].hist(data[i])
    sns.distplot(data[i],ax=axs[counter])
    axs[counter].set_ylabel("Distribution")
#     plt.xticks(rotation=90)
    axs[counter].set_xticklabels(data[i].unique(),rotation=90)
    counter+=1  

#### Conclusion

* Some of the features show bimodal, it would be lovely to learn how to treat that?

PS: I did read online, but I would prefer the kaggle community solutions. It tends to be more pragmatic and industrial, compared to some of the material I have read, which seems research-ish 


As suggested by these thereads, that the best solutions would be to use "gausian mixture" and "logistic Regression"
https://www.researchgate.net/post/How_to_analyse_a_Bimodal_response_variable
https://www.researchgate.net/post/How_can_we_deal_with_bimodal_variables

which you will see later do yield the best results. The two highest accuracy models are the Naive Bayes and logistic Regression!

But is there any solution to treat the data, logiacally I dont think so, Help Kagglers?

In [ ]:
fig, axs = plt.subplots(1,10,figsize=(30,5))
fig.subplots_adjust(hspace = .5, wspace=.5)
axs = axs.ravel()
counter=0

for i in data.columns[1:10]:

    
    data_pivot = data[['target']]
    data_pivot[i] = pd.qcut(data[i],q=10,duplicates='drop')
    data_plot = pd.pivot_table(data_pivot,'target',i)

    plt.title(i)
    sns.barplot(data_plot.index,'target',data=data_plot,ax=axs[counter])
    plt.ylabel("Percentage of Approval")
    plt.xticks(rotation=90)

    counter+=1   

#### Observations

* We can observe higher transactions (1s) for 10% of the people in most of the graphs
 
* Either they are in the begining, ie, 10th percecntile or in the end, ie, 90th percentile

* Such distinctions across all variables will yield a good accuracy

#### Ploting afer using minmax just to check what happens to the bimodals

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X = data.drop('target',axis=1)
y = data.target

mms = MinMaxScaler()
X = pd.DataFrame(mms.fit_transform(X),columns=X.columns)

data_plot = pd.concat([X,y],axis=1)

In [ ]:
fig, axs = plt.subplots(1,10,figsize=(30,5))
fig.subplots_adjust(hspace = .5, wspace=.5)
axs = axs.ravel()

counter=0
for i in data.columns[1:10]:
    
    axs[counter].set_title(i)
#     axs[counter].hist(data[i])
    sns.distplot(data_plot[i],ax=axs[counter])
    axs[counter].set_ylabel("Distribution")
#     plt.xticks(rotation=90)
    axs[counter].set_xticklabels(data_plot[i].unique(),rotation=90)
    counter+=1  

### Using Logistic Regression 

In [ ]:
# data = pd.read_csv('Data/train.csv',index_col='ID_code')
data = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/train.csv')

# data.set_index('ID_code',inplace=True)
data.drop('ID_code',axis=1,inplace=True)

In [ ]:
X = data.drop('target',axis=1)
y = data.target

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg=LogisticRegression(max_iter=300)
logreg.fit(X_train,y_train.values.ravel())
y_pred = logreg.predict(X_test)

print('Accuracy of logistic regression classifier on test set: {:.5f}'.format(logreg.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cm = confusion_matrix(y_test, y_pred) # rows = truth, cols = prediction
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))

plt.figure(figsize = (5,4))
sns.set(font_scale=1.4)

sns.heatmap(df_cm, annot=True, fmt='g', cmap="Blues")
print("Test Data Accuracy: %0.4f" % accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

#### Conclusion
Since all the data were a continuous type using logistic regression was obious

### Min Max Scaler Transformation

Since I am using Logistic Regression I want to try using MinMaxScaler() once : It might yield better result. 

The data if it is in 0-1 range it improves logistic regression results

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X = data.drop('target',axis=1)
y = data.target

mms = MinMaxScaler()
X = pd.DataFrame(mms.fit_transform(X),columns=X.columns)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg=LogisticRegression(max_iter=300)
logreg.fit(X_train,y_train.values.ravel())
y_pred = logreg.predict(X_test)

print('Accuracy of logistic regression classifier on test set: {:.5f}'.format(logreg.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cm = confusion_matrix(y_test, y_pred) # rows = truth, cols = prediction
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))

plt.figure(figsize = (5,4))
sns.set(font_scale=1.4)

sns.heatmap(df_cm, annot=True, fmt='g', cmap="Blues")
print("Test Data Accuracy: %0.4f" % accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

We can see an increase in overall accuracy by 0.01.

## Changing sample ratios by using upscaling and down scaling 

In [ ]:
data.target.value_counts()

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from collections import Counter


under = RandomUnderSampler(sampling_strategy=0.25)
over = SMOTE(sampling_strategy=0.5)

X = data.drop('target',axis=1)
y = data.target


In [ ]:
X,y = under.fit_resample(X,y)

In [ ]:
y.value_counts()

In [ ]:
X,y = over.fit_resample(X,y)

In [ ]:
y.value_counts()

In [ ]:
# summarize the new class distribution
counter = Counter(y)
print(counter)


In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg2=LogisticRegression(max_iter=300)
logreg2.fit(X_train,y_train.values.ravel())
y_pred = logreg2.predict(X_test)

print('Accuracy of logistic regression classifier on test set: {:.5f}'.format(logreg2.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cm = confusion_matrix(y_test, y_pred) # rows = truth, cols = prediction
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))

plt.figure(figsize = (5,4))
sns.set(font_scale=1.4)

sns.heatmap(df_cm, annot=True, fmt='g', cmap="Blues")
print("Test Data Accuracy: %0.4f" % accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

#### Conclusion of Udersampling and over Sampling

We can see a reduction in the prediction ratios, that is we are overpredicting the amount of 1s, after sample manuplation, thus we shall not use it!

### Min Max Scaler Transformation + Data Selection

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X = data.drop('target',axis=1)
y = data.target

mms = MinMaxScaler()
X = pd.DataFrame(mms.fit_transform(X),columns=X.columns)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

How to improve the overall accuracy of the model, 

Let us first understand : We have high recall for 0 and slightly lower precision for 0, that means we need to reduce the number of 0 predictions. 

That inturn will drive my model prediction of 1s to increase, 

* Genrally 
> * I want to reduce the elements with the lower coeff magnituge  
> * Reduce elements with high p-val (>0.02) (I am chosing a confidence level of 98%) 
* For this model
> * But I shall keep the values with high coeff magnitude if it is +ve, and Pval <0.05 -> coz the model has low 
> * I shall remove the values with coeff of magnitude -ve and pval>0.02

In [ ]:
import statsmodels.api  as sm

logit_model2=sm.Logit(y_train,X_train)
result=logit_model2.fit()
print(result.summary2())

#### First I will eleminate variables only with -ve coeff. of variables : 

In [ ]:
X_train_negd = X_train.drop(['var_7','var_10','var_17','var_29','var_30','var_27',
                             'var_38','var_39','var_41','var_73','var_98','var_61',
                             'var_100','var_103','var_136','var_153','var_158',
                             'var_161',
                             ],axis=1)

In [ ]:
X_test_negd = X_test.drop(['var_7','var_10','var_17','var_29','var_30','var_27',
                            'var_38','var_39','var_41','var_73','var_98','var_61',
                            'var_100','var_103','var_136','var_153','var_158',
                            'var_161',
                             ],axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg=LogisticRegression(max_iter=300)
logreg.fit(X_train_negd,y_train.values.ravel())
y_pred = logreg.predict(X_test_negd)

print('Accuracy of logistic regression classifier on test set: {:.5f}'.format(logreg.score(X_test_negd, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cm = confusion_matrix(y_test, y_pred) # rows = truth, cols = prediction
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))

plt.figure(figsize = (5,4))
sns.set(font_scale=1.4)

sns.heatmap(df_cm, annot=True, fmt='g', cmap="Blues")
print("Test Data Accuracy: %0.4f" % accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

##### Slight drop in recall for 1, mean a value that was 1 got predicted as 0

#### Second, I will eleminate both variables only with -ve & +ve coeff. of variables : 

In [ ]:
X_train_negd = X_train.drop(['var_7','var_10','var_17','var_29','var_30','var_27',
                             'var_38','var_39','var_41','var_73','var_98','var_61',
                             'var_100','var_103','var_136','var_153','var_158',
                             'var_161','var_183',
                             'var_16','var_37','var_46','var_65','var_69','var_79',
                             'var_96','var_117','var_124','var_126','var_185','var_189',
                             'var_61','var_60','var_47'                             
                             ],axis=1)

In [ ]:
X_test_negd = X_test.drop(['var_7','var_10','var_17','var_29','var_30','var_27',
                            'var_38','var_39','var_41','var_73','var_98','var_61',
                            'var_100','var_103','var_136','var_153','var_158',
                            'var_161','var_183',
                           'var_16','var_37','var_46','var_65','var_69','var_79',
                             'var_96','var_117','var_124','var_126','var_185','var_189',
                           'var_61','var_60','var_47'
                             ],axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg=LogisticRegression(max_iter=300)
logreg.fit(X_train_negd,y_train.values.ravel())
y_pred = logreg.predict(X_test_negd)

print('Accuracy of logistic regression classifier on test set: {:.5f}'.format(logreg.score(X_test_negd, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cm = confusion_matrix(y_test, y_pred) # rows = truth, cols = prediction
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))

plt.figure(figsize = (5,4))
sns.set(font_scale=1.4)

sns.heatmap(df_cm, annot=True, fmt='g', cmap="Blues")
print("Test Data Accuracy: %0.4f" % accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

##### Slight drop in recall for 1, mean a value that was 1 got predicted as 0

#### Using RFE : For Feature Selection

In [ ]:
# Recursive Feature Elimination
from sklearn.feature_selection import RFE

logreg = LogisticRegression(max_iter=4000)

# Select Best X Features
rfe = RFE(logreg, n_features_to_select=None)
rfe = rfe.fit(X_train, y_train.values.ravel())

In [ ]:
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
X_train_rfe = X_train[X_train.columns[rfe.support_]]
X_test_rfe = X_test[X_train.columns[rfe.support_]]

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg=LogisticRegression(max_iter=300)
logreg.fit(X_train_rfe,y_train.values.ravel())
y_pred = logreg.predict(X_test_rfe)

print('Accuracy of logistic regression classifier on test set: {:.5f}'.format(logreg.score(X_test_rfe, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cm = confusion_matrix(y_test, y_pred) # rows = truth, cols = prediction
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))

plt.figure(figsize = (5,4))
sns.set(font_scale=1.4)

sns.heatmap(df_cm, annot=True, fmt='g', cmap="Blues")
print("Test Data Accuracy: %0.4f" % accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

### Trying Other Models

##### Not all models are running with so many data, So I will select a few on my on. As we have observed, that adjusting the ratios of sample sets are imoacting the accuracy of the model, I have decieded to maintain the same ratios. 

More enlightened folks please, I implore you to give me your sugesstion and valueble feedback on the limitations of my actions!!

And also feel free to critize the hell out of this notebook and help me out :P

Please also let me know if there is a package that would have helped me select a few data points out of these 

In [ ]:
ratio = y[y==1].shape[0]/y[y==0].shape[0]
index_1 = y[y==1].index
index_0 = y[y==0].index

In [ ]:
from numpy import random

index_0_sel = random.choice(index_1, size = 20000)
index_1_sel = random.choice(index_0, size = int(20000*ratio))
index_sel = np.concatenate((index_0_sel,index_1_sel))


X_sel = X.iloc[index_sel]
y_sel = y.iloc[index_sel]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_sel, y_sel, test_size=0.33, random_state=42)

In [ ]:
X_train = X_train.drop(['var_7','var_10','var_17','var_29','var_30','var_27',
                             'var_38','var_39','var_41','var_73','var_98','var_61',
                             'var_100','var_103','var_136','var_153','var_158',
                             'var_161','var_183',
                             'var_16','var_37','var_46','var_65','var_69','var_79',
                             'var_96','var_117','var_124','var_126','var_185','var_189',
                             'var_61','var_60','var_47'                             
                             ],axis=1)

In [ ]:
X_test = X_test.drop(['var_7','var_10','var_17','var_29','var_30','var_27',
                            'var_38','var_39','var_41','var_73','var_98','var_61',
                            'var_100','var_103','var_136','var_153','var_158',
                            'var_161','var_183',
                           'var_16','var_37','var_46','var_65','var_69','var_79',
                             'var_96','var_117','var_124','var_126','var_185','var_189',
                           'var_61','var_60','var_47'
                             ],axis=1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
## Logistic Regression
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0, penalty = 'l2')
classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

results = pd.DataFrame([['Logistic Regression (Lasso)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

In [ ]:
## SVM (Linear)
from sklearn.svm import SVC
classifier = SVC(random_state = 0, kernel = 'linear', probability= True)
classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['SVM (Linear)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

results = results.append(model_results, ignore_index = True)

In [ ]:
## SVM (rbf)
from sklearn.svm import SVC
classifier = SVC(random_state = 0, kernel = 'rbf', probability= True)
classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['SVM (RBF)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

results = results.append(model_results, ignore_index = True)

In [ ]:
## Naive Bayes
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['Naive Bayes (Gaussian)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

results = results.append(model_results, ignore_index = True)

In [ ]:
## Decision Tree
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
classifier.fit(X_train, y_train)

#Predicting the best set result
y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['Decision Tree', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

results = results.append(model_results, ignore_index = True)

In [ ]:
## Random Forest Gini (n=100)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state = 0, n_estimators = 100,
                                    criterion = 'gini')
classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['Random Forest Gini (n=100)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

results = results.append(model_results, ignore_index = True)

In [ ]:
## Random Forest Gini (n=200)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state = 0, n_estimators = 200,
                                    criterion = 'gini')
classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['Random Forest Gini (n=200)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

results = results.append(model_results, ignore_index = True)

In [ ]:
## Random Forest Gini (n=300)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state = 0, n_estimators = 300,
                                    criterion = 'gini')
classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['Random Forest Gini (n=300)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

results = results.append(model_results, ignore_index = True)

In [ ]:
## Random Forest Entropy (n=100)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state = 0, n_estimators = 100,
                                    criterion = 'entropy')
classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['Random Forest Entropy (n=100)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

results = results.append(model_results, ignore_index = True)

In [ ]:
## Random Forest Entropy (n=200)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state = 0, n_estimators = 200,
                                    criterion = 'entropy')
classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['Random Forest Entropy (n=200)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

results = results.append(model_results, ignore_index = True)

In [ ]:
## Random Forest Entropy (n=300)
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state = 0, n_estimators = 300,
                                    criterion = 'entropy')
classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['Random Forest Entropy (n=300)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

results = results.append(model_results, ignore_index = True)

In [ ]:
## K-Nearest Neighbors (K-NN)
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=15, metric='minkowski', p= 2)
classifier.fit(X_train, y_train)

# Predicting Test Set
y_pred = classifier.predict(X_test)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

model_results = pd.DataFrame([['K-Nearest Neighbors (minkowski)', acc, prec, rec, f1]],
               columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

results = results.append(model_results, ignore_index = True)

### Results

In [ ]:
results

### Naive Bayes (Gaussian)

Using the full data

In [ ]:
X = data.drop('target',axis=1)
y = data.target

In [ ]:
mms = MinMaxScaler()
X = pd.DataFrame(mms.fit_transform(X),columns=X.columns)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train = X_train.drop(['var_7','var_10','var_17','var_29','var_30','var_27',
                             'var_38','var_39','var_41','var_73','var_98','var_61',
                             'var_100','var_103','var_136','var_153','var_158',
                             'var_161','var_183',
                             'var_16','var_37','var_46','var_65','var_69','var_79',
                             'var_96','var_117','var_124','var_126','var_185','var_189',
                             'var_61','var_60','var_47'                             
                             ],axis=1)

In [ ]:
X_test = X_test.drop(['var_7','var_10','var_17','var_29','var_30','var_27',
                            'var_38','var_39','var_41','var_73','var_98','var_61',
                            'var_100','var_103','var_136','var_153','var_158',
                            'var_161','var_183',
                           'var_16','var_37','var_46','var_65','var_69','var_79',
                             'var_96','var_117','var_124','var_126','var_185','var_189',
                           'var_61','var_60','var_47'
                             ],axis=1)

In [ ]:
## Naive Bayes
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print('Accuracy of Naive bayes classifier on test set: {:.5f}'.format(accuracy_score(y_test,y_pred)))

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

cm = confusion_matrix(y_test, y_pred) # rows = truth, cols = prediction
df_cm = pd.DataFrame(cm, index = (0, 1), columns = (0, 1))

plt.figure(figsize = (5,4))
sns.set(font_scale=1.4)

sns.heatmap(df_cm, annot=True, fmt='g', cmap="Blues")
print("Test Data Accuracy: %0.4f" % accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

### Test Data

In [ ]:
test = pd.read_csv("/kaggle/input/santander-customer-transaction-prediction/test.csv",index_col='ID_code')

test = test.drop(['var_7','var_10','var_17','var_29','var_30','var_27',
                             'var_38','var_39','var_41','var_73','var_98','var_61',
                             'var_100','var_103','var_136','var_153','var_158',
                             'var_161','var_183',
                             'var_16','var_37','var_46','var_65','var_69','var_79',
                             'var_96','var_117','var_124','var_126','var_185','var_189',
                             'var_61','var_60','var_47'                             
                             ],axis=1)

sol = pd.DataFrame(test.index)

mms = MinMaxScaler()
test = pd.DataFrame(mms.fit_transform(test),columns=test.columns)

sol['target']=classifier.predict(test)

sol.set_index('ID_code',inplace=True)

sol.to_csv('submission_NB')

# Conslusion

Some Advice on how should I proceed to  improve my accuracy?